### imports, set up

In [5]:
from datasets import load_dataset
import os
from miditoolkit import MidiFile
from miditok import REMI
import torch
from transformers import (
    AutoModelForCausalLM,
    Trainer,
    default_data_collator,
    set_seed,
)
from itertools import chain

In [21]:
max_train_samples = 10000
block_size = 1024

# esta signature no permite procesamiento por batches
def midifn2tokens(x, data_dir, tokenizer, return_dict=True):
    x['midi_filename'] = os.path.join(data_dir, x['midi_filename'])
    midi_file = MidiFile(x['midi_filename'])
    tokens = tokenizer.midi_to_tokens(midi_file)[0]
    x['input_ids'] = tokens
    if return_dict:
        return x
    else:
        return tokens

DATA_DIR = "/Users/juanigp/Desktop/data/e-gmd-v1.0.0"
csv_dir = "e-gmd-v1.0.0.csv"
csv_dir = os.path.join(DATA_DIR, csv_dir)
dataset = load_dataset('csv', data_files=csv_dir)
dataset = dataset['train'].train_test_split(test_size = 0.2)
train_dataset = dataset['train'] 
train_dataset = train_dataset.select(range(max_train_samples))# for quick testing
tokenizer = REMI()
column_names = train_dataset.column_names

map_func = lambda x: midifn2tokens(x, DATA_DIR, tokenizer, True)
train_dataset = train_dataset.map(
    map_func, 
    num_proc=8,
    # batched=True,  
    remove_columns=column_names, # al hacer esto es mas facil agrupar por chunks luego
)

Using custom data configuration default-51ad44df56849337
Reusing dataset csv (/Users/juanigp/.cache/huggingface/datasets/csv/default-51ad44df56849337/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)
#0:   0%|                                              | 0/1250 [00:00<?, ?ex/s]



#3:   0%|                                              | 0/1250 [00:00<?, ?ex/s]

#2:   0%|                                              | 0/1250 [00:00<?, ?ex/s]



#4:   0%|                                              | 0/1250 [00:00<?, ?ex/s]




#5:   0%|                                              | 0/1250 [00:00<?, ?ex/s]





#6:   0%|                                              | 0/1250 [00:00<?, ?ex/s]






#0:   1%|▎                                    | 12/1250 [00:00<00:14, 87.00ex/s]


#3:   0%|                                      | 3/1250 [00:00<00:50, 24.63ex/s]




#5:   1%|▏                                     | 8/1250 [00:00<00:15, 79.61ex/s]

#2:   0%|           

#6:   7%|██▋                                  | 89/1250 [00:02<00:40, 28.90ex/s]

#2:   7%|██▌                                  | 86/1250 [00:02<00:30, 37.58ex/s]






#7:   6%|██▏                                  | 74/1250 [00:02<00:35, 33.40ex/s]


#3:   5%|█▋                                   | 57/1250 [00:02<00:45, 26.47ex/s]



#4:   9%|███                                 | 108/1250 [00:02<00:24, 45.95ex/s]






#7:   6%|██▎                                  | 80/1250 [00:02<00:30, 38.52ex/s]




#1:  10%|███▌                                | 123/1250 [00:02<00:20, 54.07ex/s]

#2:   8%|██▊                                  | 95/1250 [00:02<00:25, 44.50ex/s]



#4:   9%|███▎                                | 114/1250 [00:02<00:23, 48.48ex/s]






#7:   7%|██▌                                  | 88/1250 [00:02<00:24, 47.57ex/s]





#0:  11%|████                                | 143/1250 [00:02<00:27, 40.21ex/s]

#2:   8%|██▉                                 | 100/1250 [00:02<00:26, 4

#5:  14%|████▉                               | 173/1250 [00:05<00:25, 42.62ex/s]


#1:  18%|██████▍                             | 224/1250 [00:05<00:31, 32.29ex/s]

#2:  16%|█████▋                              | 198/1250 [00:05<00:30, 34.48ex/s]




#5:  14%|█████▏                              | 180/1250 [00:05<00:22, 46.72ex/s]


#3:  11%|███▉                                | 137/1250 [00:05<00:22, 50.41ex/s]






#1:  18%|██████▋                             | 231/1250 [00:05<00:27, 36.47ex/s]




#5:  15%|█████▌                              | 191/1250 [00:05<00:18, 57.95ex/s]


#3:  12%|████▏                               | 144/1250 [00:05<00:21, 50.60ex/s]






#7:  14%|█████                               | 174/1250 [00:05<00:33, 31.90ex/s]

#2:  16%|█████▊                              | 203/1250 [00:05<00:31, 32.72ex/s]





#6:  15%|█████▍                              | 188/1250 [00:05<01:02, 17.00ex/s]



#1:  19%|██████▊                             | 237/1250 [00:05<00:26, 37.

#1:  24%|████████▋                           | 302/1250 [00:07<00:23, 40.49ex/s]

#2:  24%|████████▋                           | 301/1250 [00:07<00:24, 38.35ex/s]


#3:  20%|███████▎                            | 253/1250 [00:07<00:30, 32.33ex/s]




#5:  24%|████████▊                           | 306/1250 [00:07<00:19, 47.39ex/s]





#6:  20%|███████▎                            | 254/1250 [00:07<00:57, 17.46ex/s]



#0:  25%|█████████                           | 314/1250 [00:08<00:30, 30.67ex/s]




#5:  25%|█████████▏                          | 317/1250 [00:08<00:16, 57.21ex/s]


#3:  21%|███████▋                            | 265/1250 [00:08<00:22, 42.99ex/s]





#6:  21%|███████▍                            | 257/1250 [00:08<00:53, 18.54ex/s]

#2:  25%|████████▉                           | 311/1250 [00:08<00:23, 40.27ex/s]



#0:  26%|█████████▎                          | 324/1250 [00:08<00:22, 40.63ex/s]

#1:  25%|█████████                           | 315/1250 [00:08<00:25, 37.38ex/

#7:  30%|██████████▉                         | 378/1250 [00:10<00:24, 35.32ex/s]



#0:  32%|███████████▍                        | 397/1250 [00:10<00:43, 19.48ex/s]




#5:  30%|██████████▉                         | 379/1250 [00:10<00:30, 28.98ex/s]

#2:  32%|███████████▌                        | 402/1250 [00:10<00:17, 48.64ex/s]

#0:  33%|███████████▊                        | 412/1250 [00:10<00:25, 32.66ex/s]





#6:  26%|█████████▌                          | 330/1250 [00:10<00:23, 39.89ex/s]






#7:  31%|███████████                         | 383/1250 [00:10<00:32, 26.79ex/s]




#5:  31%|███████████                         | 384/1250 [00:10<00:32, 26.35ex/s]

#1:  34%|████████████                        | 419/1250 [00:10<00:13, 59.57ex/s]





#6:  27%|█████████▊                          | 342/1250 [00:10<00:17, 53.27ex/s]






#7:  32%|███████████▍                        | 398/1250 [00:10<00:18, 45.32ex/s]



#4:  29%|██████████▍                         | 363/1250 [00:10<00:31, 

...

#2:  67%|████████████████████████▎           | 843/1250 [00:23<00:08, 48.75ex/s]





#6:  68%|████████████████████████▍           | 848/1250 [00:23<00:07, 54.35ex/s]


#3:  76%|███████████████████████████▍        | 953/1250 [00:23<00:08, 36.42ex/s]




#5:  65%|███████████████████████▍            | 812/1250 [00:23<00:10, 41.43ex/s]






#0:  73%|██████████████████████████          | 907/1250 [00:23<00:14, 24.29ex/s]



#4:  67%|████████████████████████▏           | 839/1250 [00:23<00:09, 44.64ex/s]






#0:  73%|██████████████████████████▎         | 912/1250 [00:23<00:13, 24.77ex/s]




#5:  65%|███████████████████████▌            | 818/1250 [00:23<00:11, 37.29ex/s]



#4:  68%|████████████████████████▎           | 846/1250 [00:23<00:09, 43.62ex/s]

#1:  67%|████████████████████████▏           | 840/1250 [00:23<00:09, 42.23ex/s]






#1:  68%|████████████████████████▎           | 845/1250 [00:23<00:09, 43.41ex/s]

#2:  69%|████████████████████████▉           | 867/1250 [00:23<00:07

#1:  74%|██████████████████████████▊         | 931/1250 [00:25<00:11, 26.88ex/s]




#5:  71%|█████████████████████████▋          | 890/1250 [00:25<00:17, 21.08ex/s]






#1:  76%|███████████████████████████▎        | 948/1250 [00:25<00:06, 47.09ex/s]


#3:  85%|█████████████████████████████▊     | 1066/1250 [00:25<00:05, 34.05ex/s]





#6:  77%|███████████████████████████▊        | 965/1250 [00:25<00:07, 39.80ex/s]



#4:  75%|██████████████████████████▉         | 937/1250 [00:26<00:07, 44.21ex/s]

#2:  77%|███████████████████████████▊        | 966/1250 [00:26<00:06, 42.64ex/s]




#1:  78%|████████████████████████████▏       | 977/1250 [00:26<00:03, 75.90ex/s]



#4:  76%|███████████████████████████▌        | 955/1250 [00:26<00:04, 60.37ex/s]





#6:  78%|███████████████████████████▉        | 971/1250 [00:26<00:07, 37.06ex/s]






#7:  76%|███████████████████████████▎        | 949/1250 [00:26<00:08, 35.00ex/s]


#3:  86%|█████████████████████████████▉     | 1071/1250 [00:26<00:05

#3:  90%|███████████████████████████████▍   | 1124/1250 [00:28<00:03, 34.93ex/s]






#1:  85%|█████████████████████████████▊     | 1066/1250 [00:28<00:03, 56.90ex/s]





#6:  85%|█████████████████████████████▋     | 1062/1250 [00:28<00:04, 37.81ex/s]




#0:  85%|█████████████████████████████▊     | 1065/1250 [00:28<00:06, 26.73ex/s]




#5:  81%|████████████████████████████▍      | 1015/1250 [00:28<00:04, 49.83ex/s]


#1:  86%|██████████████████████████████     | 1074/1250 [00:28<00:03, 54.06ex/s]





#6:  85%|█████████████████████████████▉     | 1067/1250 [00:28<00:05, 35.13ex/s]

#0:  86%|█████████████████████████████▉     | 1070/1250 [00:28<00:06, 26.94ex/s]


#3:  91%|███████████████████████████████▊   | 1135/1250 [00:28<00:03, 32.06ex/s]

#2:  86%|██████████████████████████████▏    | 1079/1250 [00:28<00:03, 53.79ex/s]






#7:  81%|████████████████████████████▎      | 1011/1250 [00:28<00:17, 13.85ex/s]




#5:  82%|████████████████████████████▌      | 1021/1250 [00:29<00:05,

#2:  91%|████████████████████████████████   | 1143/1250 [00:31<00:02, 43.64ex/s]






#7:  89%|███████████████████████████████    | 1109/1250 [00:31<00:02, 48.36ex/s]




#0:  94%|████████████████████████████████▊  | 1172/1250 [00:31<00:02, 33.01ex/s]



#4:  92%|████████████████████████████████   | 1146/1250 [00:31<00:02, 35.64ex/s]


#3:  98%|██████████████████████████████████▏| 1221/1250 [00:31<00:00, 35.60ex/s]





#1:  91%|███████████████████████████████▋   | 1132/1250 [00:31<00:04, 24.69ex/s]

#2:  92%|████████████████████████████████▏  | 1149/1250 [00:31<00:02, 37.33ex/s]





#6:  94%|████████████████████████████████▋  | 1169/1250 [00:31<00:01, 57.73ex/s]




#5:  90%|███████████████████████████████▌   | 1128/1250 [00:31<00:03, 40.56ex/s]






#1:  91%|███████████████████████████████▊   | 1136/1250 [00:31<00:04, 26.10ex/s]

#2:  92%|████████████████████████████████▎  | 1155/1250 [00:31<00:02, 38.65ex/s]



#4:  92%|████████████████████████████████▎  | 1155/1250 [00:31<00:02,

#7: 100%|███████████████████████████████████| 1250/1250 [00:34<00:00, 36.66ex/s]


In [38]:
# Main data processing function that will concatenate all texts from our dataset and generate chunks of block_size.
def group_seqs(examples):
    # Concatenate all texts.
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

train_dataset = train_dataset.map(
    group_seqs, 
    num_proc=8,
    batched=True,
)

#1:   0%|                                                 | 0/2 [00:00<?, ?ba/s]

#2:   0%|                                                 | 0/2 [00:00<?, ?ba/s]



#4:   0%|                                                 | 0/2 [00:00<?, ?ba/s]


#3:   0%|                                                 | 0/2 [00:00<?, ?ba/s]





#6:   0%|                                                 | 0/2 [00:00<?, ?ba/s]




#5:   0%|                                                 | 0/2 [00:00<?, ?ba/s]






#1:  50%|████████████████████▌                    | 1/2 [00:00<00:00,  1.50ba/s]


#3:  50%|████████████████████▌                    | 1/2 [00:00<00:00,  1.53ba/s]



#4:  50%|████████████████████▌                    | 1/2 [00:00<00:00,  1.42ba/s]

#0:  50%|████████████████████▌                    | 1/2 [00:00<00:00,  1.31ba/s]





#6:  50%|████████████████████▌                    | 1/2 [00:00<00:00,  1.36ba/s]






#7:  50%|████████████████████▌                    | 1/2 [00:00<00:00,  

In [39]:
model = AutoModelForCausalLM.from_pretrained('sshleifer/tiny-gpt2')
model.resize_token_embeddings(len(tokenizer.vocab))
# model.get_input_embeddings()

Embedding(218, 2)

In [40]:
trainer = Trainer(
    model=model,
    train_dataset=train_dataset
    # Data collator will default to DataCollatorWithPadding, so we change it.
    # data_collator=default_data_collator,
)

In [41]:
trainer.train()

***** Running training *****
  Num examples = 11098
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 4164


Step,Training Loss
500,5.342100
1000,5.276000
1500,5.218000
2000,5.167400
2500,5.125700
3000,5.092800
3500,5.070300
4000,5.057600


Saving model checkpoint to tmp_trainer/checkpoint-500
Configuration saved in tmp_trainer/checkpoint-500/config.json
Model weights saved in tmp_trainer/checkpoint-500/pytorch_model.bin
Saving model checkpoint to tmp_trainer/checkpoint-1000
Configuration saved in tmp_trainer/checkpoint-1000/config.json
Model weights saved in tmp_trainer/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to tmp_trainer/checkpoint-1500
Configuration saved in tmp_trainer/checkpoint-1500/config.json
Model weights saved in tmp_trainer/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to tmp_trainer/checkpoint-2000
Configuration saved in tmp_trainer/checkpoint-2000/config.json
Model weights saved in tmp_trainer/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to tmp_trainer/checkpoint-2500
Configuration saved in tmp_trainer/checkpoint-2500/config.json
Model weights saved in tmp_trainer/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to tmp_trainer/checkpoint-3000
Configuration

TrainOutput(global_step=4164, training_loss=5.164203669450011, metrics={'train_runtime': 1280.0138, 'train_samples_per_second': 26.011, 'train_steps_per_second': 3.253, 'total_flos': 31092867072.0, 'train_loss': 5.164203669450011, 'epoch': 3.0})

### to do: 
- funciones computar metricas
- callback guardar modelos
- guardar metricas training
- eval loop y guardar metricas
- __script + hydra__
- data parallelism?
- guardar dataset preprocesado? __load_from_cache_file__ en dataset.map